 # ORB Detector Parameter Tuning 
## Setup the required functions and input images

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  1 20:06:06 2019

@author: vik748
"""
import numpy as np
import cv2
import sys
from matplotlib import pyplot as plt
from ssc import *
%matplotlib inline
from scipy import spatial
from mahotas import features

def draw_keypoints(vis_orig, keypoints, color = (255, 255, 0), thick = 1):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.circle(vis, (int(x), int(y)), int(vis.shape[1]/200), color, thickness=thick)
    return vis

def draw_markers(vis_orig, keypoints, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.drawMarker(vis, (int(x), int(y)), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

def draw_marker_pts(vis_orig, pts, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for row in pts[:,0,:]:
        cv2.drawMarker(vis, (int(row[0]), int(row[1])), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

def draw_arrows(vis_orig, points1, points2, color = (0, 255, 0), thick = 1):
    vis = cv2.cvtColor(vis_orig,cv2.COLOR_GRAY2RGB)
    #rad = int(vis.shape[1]/200)
    for p1,p2 in zip(points1,points2):
        cv2.arrowedLine(vis, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), color=(0,255,0), thickness=thick)
    return vis

def draw_feature_tracks(img_left,kp1,img_right,kp2, matches, mask, display_invalid=False, color=(0, 255, 0)):
    '''
    This function extracts takes a 2 images, set of keypoints and a mask of valid
    (mask as a ndarray) keypoints and plots the valid ones in green and invalid in red.
    The mask should be the same length as matches
    '''
    if mask is None: bool_mask = np.ones((len(matches)), dtype=bool)
    else: bool_mask = mask.astype(bool)
    valid_right_matches = np.array([kp2[mat.trainIdx].pt for is_match, mat in zip(bool_mask, matches) if is_match])
    valid_left_matches = np.array([kp1[mat.queryIdx].pt for is_match, mat in zip(bool_mask, matches) if is_match])
    #img_right_out = draw_points(img_right, valid_right_matches)
    img_right_out = draw_arrows(img_right, valid_left_matches, valid_right_matches)
    return img_right_out

def displayMatches(img_left,kp1,img_right,kp2, matches, mask, display_invalid, in_image=None, color=(0, 255, 0)):
    '''
    This function extracts takes a 2 images, set of keypoints and a mask of valid
    (mask as a ndarray) keypoints and plots the valid ones in green and invalid in red.
    The mask should be the same length as matches
    '''
    bool_mask = mask.astype(bool)
    if in_image is None: mode_flag=0
    else: mode_flag =1
    img_valid = cv2.drawMatches(img_left,kp1,img_right,kp2,matches, in_image, 
                                matchColor=color, 
                                matchesMask=bool_mask.ravel().tolist(), flags=mode_flag)
    
    if display_invalid:
        img_valid = cv2.drawMatches(img_left,kp1,img_right,kp2,matches, img_valid, 
                                  matchColor=(255, 0, 0), 
                                  matchesMask=np.invert(bool_mask).ravel().tolist(), 
                                  flags=1)
    return img_valid


K = np.array([[699.33112889, 0.0, 403.23876197],
              [0.0, 693.66457792, 293.40739086],
              [0.0, 0.0, 1.0]])

D =  np.array([-2.78089511e-01,  1.30037134e-01, -1.17555797e-04, -1.55337290e-04, -4.34486330e-02])

if sys.platform == 'darwin':
    path = '/Users/vik748/Google Drive/'
else:
    path = '/home/vik748/'
img1 = cv2.imread(path+'data/time_lapse_5_cervino_800x600/G0057821.png',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE
img2 = cv2.imread(path+'data/time_lapse_5_cervino_800x600/G0057826.png',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE


gr1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gr2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(4,3))
gr1 = clahe.apply(gr1)
gr2 = clahe.apply(gr2)


# create a mask image filled with 1s, the size of original image
m1 = cv2.imread(path+'data/time_lapse_5_cervino_800x600_masks_out/G0057821_mask.png',cv2.IMREAD_GRAYSCALE)
m2 = cv2.imread(path+'data/time_lapse_5_cervino_800x600_masks_out/G0057826_mask.png',cv2.IMREAD_GRAYSCALE)

def radial_non_max(kp_list, dist):
    kp = np.array(kp_list)
    kp_mask = np.ones(len(kp), dtype=bool)
    pts = [k.pt for k in kp]
    tree = spatial.cKDTree(pts)
    #print ("len of kp1:",len(kp))
    for i, k in enumerate(kp):
        if kp_mask[i]:
            pt = tree.data[i]
            idx = tree.query_ball_point(tree.data[i], dist, p=2., eps=0, n_jobs=1)
            resp = [kp[ii].response for ii in idx]
            _, maxi = max([(v,i) for i,v in enumerate(resp)])
            del idx[maxi]
            for kp_i in idx:
                kp_mask[kp_i] = False 
    return kp[kp_mask].tolist()

def bounding_box(points, min_x=-np.inf, max_x=np.inf, min_y=-np.inf,
                        max_y=np.inf):
    """ Compute a bounding_box filter on the given points

    Parameters
    ----------                        
    points: (n,2) array
        The array containing all the points's coordinates. Expected format:
            array([
                [x1,y1],
                ...,
                [xn,yn]])

    min_i, max_i: float
        The bounding box limits for each coordinate. If some limits are missing,
        the default values are -infinite for the min_i and infinite for the max_i.

    Returns
    -------
    bb_filter : boolean array
        The boolean mask indicating wherever a point should be keept or not.
        The size of the boolean mask will be the same as the number of given points.

    """

    bound_x = np.logical_and(points[:, 0] > min_x, points[:, 0] < max_x)
    bound_y = np.logical_and(points[:, 1] > min_y, points[:, 1] < max_y)

    bb_filter = np.logical_and(bound_x, bound_y)

    return bb_filter


def tiled_features(kp, img_shape, tiley, tilex):
    feat_per_cell = int(len(kp)/(tilex*tiley))
    HEIGHT, WIDTH = img_shape
    assert WIDTH%tiley == 0, "Width is not a multiple of tilex"
    assert HEIGHT%tilex == 0, "Height is not a multiple of tiley"
    w_width = int(WIDTH/tiley)
    w_height = int(HEIGHT/tilex)
        
    xx = np.linspace(0,HEIGHT-w_height,tilex,dtype='int')
    yy = np.linspace(0,WIDTH-w_width,tiley,dtype='int')
        
    kps = np.array([])
    pts = np.array([keypoint.pt for keypoint in kp])
    kp = np.array(kp)
    
    for ix in xx:
        for iy in yy:
            inbox_mask = bounding_box(pts, iy,iy+w_height,ix,ix+w_height)
            inbox = kp[inbox_mask]
            inbox_sorted = sorted(inbox, key = lambda x:x.response, reverse = True)
            inbox_sorted_out = inbox_sorted[:feat_per_cell]
            kps = np.append(kps,inbox_sorted_out)
    return kps.tolist()


## Feature Detection

In [ ]:
TILE_KP = True
RADIAL_NON_MAX = True
RADIAL_NON_MAX_RADIUS = 5
TILEY = 4
TILEX = 3

detector = cv2.ORB_create(nfeatures=10000, scaleFactor=2.0, nlevels=4, edgeThreshold=31, 
                           firstLevel=0, WTA_K=2, scoreType=cv2.ORB_HARRIS_SCORE,  
                           patchSize=31, fastThreshold=15)
'''
detector = cv2.xfeatures2d.SIFT_create(nfeatures = 10000, nOctaveLayers = 3, contrastThreshold = 0.01, 
                                       edgeThreshold = 20, sigma = 1.6)
'''
# find the keypoints and descriptors with ORB
kp1 = detector.detect(gr1,m1)
print("Detected kps: ",len(kp1))
kp2 = detector.detect(gr2,m2)

if TILE_KP:
    kp1 = tiled_features(kp1, gr1.shape, TILEY, TILEX)
    kp2 = tiled_features(kp2, gr2.shape, TILEY, TILEX)
    print ("Points after tiling supression: ",len(kp1))

if RADIAL_NON_MAX:
    kp1 = radial_non_max(kp1,RADIAL_NON_MAX_RADIUS)
    kp2 = radial_non_max(kp2,RADIAL_NON_MAX_RADIUS)
    print ("Points after radial supression: ",len(kp1))

img_out1 = draw_keypoints(cv2.cvtColor(gr1, cv2.COLOR_GRAY2BGR),kp1)
img_out2 = draw_keypoints(cv2.cvtColor(gr2, cv2.COLOR_GRAY2BGR),kp2)


fig, [ax1, ax2] = plt.subplots(1,2,dpi=200)
fig.suptitle('800x600 Yellow circle defult setting patchSize=31')
ax1.axis("off")
ax1.imshow(img_out1)
ax2.axis("off")
ax2.imshow(img_out2)
fig.subplots_adjust(0,0,1,1,0.0,0.0)
plt.show()

## Feature matching

In [ ]:
kp1, des1 = detector.compute(gr1,kp1)
kp2, des2 = detector.compute(gr2,kp2)
'''
FLANN_INDEX_LSH = 6
FLANN_INDEX_KDTREE = 1
matcher = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 20,
                                   multi_probe_level = 2), dict(checks=100))
'''
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck=True)
#matches12_knn = matcher.knnMatch(des1,des2,k=2)
matches12 = matcher.match(des1,des2)
#matches12 = [item for sublist in matches12_knn for item in sublist]
print("Found matches: ",len(matches12))

kp1_match_12 = np.array([kp1[mat.queryIdx].pt for mat in matches12])
kp2_match_12 = np.array([kp2[mat.trainIdx].pt for mat in matches12])

#matches12 = sorted(matches12, key = lambda x:x.distance)
#matches12 = matches12[:(int)(len(matches12)*.75)]

kp1_match_12_ud = cv2.undistortPoints(np.expand_dims(kp1_match_12,axis=1),K,D)
kp2_match_12_ud = cv2.undistortPoints(np.expand_dims(kp2_match_12,axis=1),K,D)

E_12, mask_e_12 = cv2.findEssentialMat(kp1_match_12_ud, kp2_match_12_ud, focal=1.0, pp=(0., 0.), 
                                       method=cv2.RANSAC, prob=0.9999, threshold=0.001)

print("After essential: ", np.sum(mask_e_12))

img_valid = draw_feature_tracks(gr1,kp1,gr2,kp2, matches12, mask_e_12, display_invalid=True, color=(0, 255, 0))
#img_valid = displayMatches(gr1,kp1,gr2,kp2, matches12, mask_e_12, display_invalid=False, color=(0, 255, 0))

fig, ax= plt.subplots(dpi=200)
plt.title('ORB detector with BF Matcher and Cheirality filter')
plt.axis("off")
plt.imshow(img_valid)
plt.show()


In [ ]:
    matches_knn = matcher.knnMatch(des1,des2, k=5)
    matches = []
    kp1_match = []
    kp2_match = []
    
    for i,match in enumerate(matches_knn):
        if len(match)>1:
            if match[0].distance < 0.80*match[1].distance:
                matches.append(match[0])
                kp1_match.append(kp1[match[0].queryIdx].pt)
                kp2_match.append(kp2[match[0].trainIdx].pt)
        elif len(match)==1:
            matches.append(match[0])
            kp1_match.append(kp1[match[0].queryIdx].pt)
            kp2_match.append(kp2[match[0].trainIdx].pt)


In [ ]:
matches12 = [item for sublist in matches_knn for item in sublist]

In [ ]:
a = features.zernike_moments(gr1, 22, 8,cm=np.array([[50.0,50.0]]))

a

In [ ]:
print (a)